In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import product
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [52]:
df = pd.read_csv('train_adj.csv')
df

,Unnamed: 0,date,ticker,firm,volume,open,high,low,close,adjustTrue
0,0,2023-05-30,A000020,동화약품,201361.0,9960.0,10040.0,9640.0,9700.0,1
1,1,2023-05-26,A000020,동화약품,196257.0,10050.0,10150.0,9850.0,9850.0,1
2,2,2023-05-25,A000020,동화약품,398326.0,9660.0,10180.0,9660.0,10040.0,1
3,3,2023-05-24,A000020,동화약품,205243.0,9770.0,9820.0,9550.0,9740.0,1
4,4,2023-05-23,A000020,동화약품,641524.0,9160.0,9900.0,9160.0,9770.0,1
...,...,...,...,...,...,...,...,...,...,...
987995,489,2021-06-07,A383800,LX홀딩스,2714980.0,10550.0,11150.0,10500.0,10800.0,1
987996,490,2021-06-04,A383800,LX홀딩스,1737593.0,10450.0,10650.0,10350.0,10450.0,1
987997,491,2021-06-03,A383800,LX홀딩스,2709800.0,10650.0,10700.0,10300.0,10400.0,1
987998,492,2021-06-02,A383800,LX홀딩스,2426922.0,10700.0,10850.0,10600.0,10700.0,1


In [53]:
# 아무 열에서 NaN 값을 하나라도 가지고 있는 행들의 "ticker" 열의 값을 출력
selected_tickers = df[df.isnull().any(axis=1)]['ticker'].unique()
print(selected_tickers)

['A006580' 'A012600' 'A015540' 'A033340' 'A065560' 'A069110' 'A078590'
 'A099520' 'A109070' 'A160600' 'A178780' 'A223310' 'A257370' 'A263540']


In [56]:
988000 - 14*494

981084

In [54]:
# 주어진 ticker 열 값을 가진 데이터를 DataFrame에서 제거(drop)
selected_tickers = ['A006580', 'A012600', 'A015540', 'A033340', 'A065560',
                    'A069110', 'A078590', 'A099520', 'A109070', 'A160600',
                    'A178780', 'A223310', 'A257370', 'A263540']

df = df[~df['ticker'].isin(selected_tickers)]
df

,Unnamed: 0,date,ticker,firm,volume,open,high,low,close,adjustTrue
0,0,2023-05-30,A000020,동화약품,201361.0,9960.0,10040.0,9640.0,9700.0,1
1,1,2023-05-26,A000020,동화약품,196257.0,10050.0,10150.0,9850.0,9850.0,1
2,2,2023-05-25,A000020,동화약품,398326.0,9660.0,10180.0,9660.0,10040.0,1
3,3,2023-05-24,A000020,동화약품,205243.0,9770.0,9820.0,9550.0,9740.0,1
4,4,2023-05-23,A000020,동화약품,641524.0,9160.0,9900.0,9160.0,9770.0,1
...,...,...,...,...,...,...,...,...,...,...
987995,489,2021-06-07,A383800,LX홀딩스,2714980.0,10550.0,11150.0,10500.0,10800.0,1
987996,490,2021-06-04,A383800,LX홀딩스,1737593.0,10450.0,10650.0,10350.0,10450.0,1
987997,491,2021-06-03,A383800,LX홀딩스,2709800.0,10650.0,10700.0,10300.0,10400.0,1
987998,492,2021-06-02,A383800,LX홀딩스,2426922.0,10700.0,10850.0,10600.0,10700.0,1


In [57]:
# 'date' 열을 날짜 형식으로 변환합니다.
df['date'] = pd.to_datetime(df['date'])

# 단계 1: 'ticker' 열로 데이터를 그룹화합니다.
grouped = df.groupby('ticker')

# 각 그룹에서 날짜를 역순으로 정렬합니다.
grouped = grouped.apply(lambda x: x.sort_values(by='date', ascending=True))
type(grouped)

pandas.core.frame.DataFrame

In [58]:
grouped

Unnamed: 0       date   ticker   firm    volume     open  \
ticker                                                                     
A000020 493            493 2021-06-01  A000020   동화약품  114966.0  14700.0   
        492            492 2021-06-02  A000020   동화약품  109559.0  14700.0   
        491            491 2021-06-03  A000020   동화약품   96158.0  14550.0   
        490            490 2021-06-04  A000020   동화약품  133900.0  14600.0   
        489            489 2021-06-07  A000020   동화약품  511140.0  14800.0   
...                    ...        ...      ...    ...       ...      ...   
A383800 987510           4 2023-05-23  A383800  LX홀딩스  150364.0   8390.0   
        987509           3 2023-05-24  A383800  LX홀딩스  122457.0   8310.0   
        987508           2 2023-05-25  A383800  LX홀딩스   84241.0   8300.0   
        987507           1 2023-05-26  A383800  LX홀딩스  126681.0   8300.0   
        987506           0 2023-05-30  A383800  LX홀딩스   70489.0   8300.0   

                   high      low    close  adjustTrue  
ticker                                                 
A000020 493     14700.0  14450.0  14600.0           1  
        492     14700.0  14450.0  14500.0           1  
        491     14650.0  14450.0  14600.0           1  
        490     14800.0  14550.0  14700.0           1  
        489     15550.0  14750.0  15150.0           1  
...                 ...      ...      ...         ...  
A383800 987510   8390.0   8310.0   8330.0           1  
        987509   8340.0   8280.0   8300.0           1  
        987508   8310.0   8270.0   8310.0           1  
        987507   8310.0   8270.0   8280.0           1  
        987506   8300.0   8270.0   8290.0           1  

[981084 rows x 10 columns]

In [59]:
# 단계 2: 각 그룹의 'close' 열을 14 위치로 이동시켜 14일 뒤의 값으로 수정합니다.
grouped['close_14_days_ahead'] = grouped['close'].groupby(level=0).shift(-14)

# 단계 3: 각 그룹에서 마지막 14개 행을 삭제합니다.
grouped = grouped.groupby(level=0).apply(lambda x: x.iloc[:-14])

# 단계 4: 인덱스를 재설정합니다.
df = grouped.reset_index(drop=True)

# 수정된 DataFrame을 출력합니다.
df

,Unnamed: 0,date,ticker,firm,volume,open,high,low,close,adjustTrue,close_14_days_ahead
0,493,2021-06-01,A000020,동화약품,114966.0,14700.0,14700.0,14450.0,14600.0,1,14500.0
1,492,2021-06-02,A000020,동화약품,109559.0,14700.0,14700.0,14450.0,14500.0,1,14450.0
2,491,2021-06-03,A000020,동화약품,96158.0,14550.0,14650.0,14450.0,14600.0,1,14550.0
3,490,2021-06-04,A000020,동화약품,133900.0,14600.0,14800.0,14550.0,14700.0,1,15400.0
4,489,2021-06-07,A000020,동화약품,511140.0,14800.0,15550.0,14750.0,15150.0,1,15400.0
...,...,...,...,...,...,...,...,...,...,...,...
953275,18,2023-05-02,A383800,LX홀딩스,67407.0,8530.0,8570.0,8520.0,8540.0,1,8330.0
953276,17,2023-05-03,A383800,LX홀딩스,132215.0,8540.0,8620.0,8520.0,8560.0,1,8300.0
953277,16,2023-05-04,A383800,LX홀딩스,71007.0,8560.0,8620.0,8560.0,8590.0,1,8310.0
953278,15,2023-05-08,A383800,LX홀딩스,163568.0,8600.0,8700.0,8600.0,8650.0,1,8280.0


In [60]:
# 첫 번째 열('Unnamed: 0')을 삭제합니다.
df.drop(columns='Unnamed: 0', inplace=True)

# 'close_14_days_ahead' 열의 이름을 'y'로 변경합니다.
df.rename(columns={'close_14_days_ahead': 'y'}, inplace=True)
df

,date,ticker,firm,volume,open,high,low,close,adjustTrue,y
0,2021-06-01,A000020,동화약품,114966.0,14700.0,14700.0,14450.0,14600.0,1,14500.0
1,2021-06-02,A000020,동화약품,109559.0,14700.0,14700.0,14450.0,14500.0,1,14450.0
2,2021-06-03,A000020,동화약품,96158.0,14550.0,14650.0,14450.0,14600.0,1,14550.0
3,2021-06-04,A000020,동화약품,133900.0,14600.0,14800.0,14550.0,14700.0,1,15400.0
4,2021-06-07,A000020,동화약품,511140.0,14800.0,15550.0,14750.0,15150.0,1,15400.0
...,...,...,...,...,...,...,...,...,...,...
953275,2023-05-02,A383800,LX홀딩스,67407.0,8530.0,8570.0,8520.0,8540.0,1,8330.0
953276,2023-05-03,A383800,LX홀딩스,132215.0,8540.0,8620.0,8520.0,8560.0,1,8300.0
953277,2023-05-04,A383800,LX홀딩스,71007.0,8560.0,8620.0,8560.0,8590.0,1,8310.0
953278,2023-05-08,A383800,LX홀딩스,163568.0,8600.0,8700.0,8600.0,8650.0,1,8280.0


In [65]:
# 전체데이터개수 - NaN인 종목들 - 14일 * NaN아닌종목들
988000 - 14*494 - 14*(2000-14)

953280

## 정상성 검정
ARIMAX 모델은 시계열 데이터가 정상성을 가지고 있어야 하며, 외부 변수(exog_columns) 역시 정상성을 가지는 것이 중요합니다.

따라서 데이터가 정상성을 가지지 않는다면 먼저 시계열 데이터를 정상성을 갖도록 전처리해야 합니다.

대표적으로 Augmented Dickey-Fuller (ADF) 검정과 Kwiatkowski-Phillips-Schmidt-Shin (KPSS) 검정이 있습니다. 이들 검정을 통해 시계열 데이터의 정상성을 확인할 수 있습니다.


- ADF
  - p-value는 귀무가설이 참일 확률을 나타내는 값으로, 시계열 데이터가 정상성을 가지는지 여부를 결정하는 데 사용됩니다. 일반적으로 p-value가 0.05보다 크면 귀무가설(정상성을 가진다)을 기각하지 않으며, 데이터가 정상성을 가질 가능성이 높다고 판단합니다. 즉, p-value가 0.05보다 크면 시계열 데이터는 정상성을 가질 가능성이 높습니다. p-value가 **0.05보다 작으면** 정상성을 가지지 않는 것으로 간주됩니다.


- KPSS
  - p-value가 0.05보다 크면 (일반적으로 0.05 또는 0.01보다 큼): 귀무가설을 기각하지 않으며, 데이터가 정상성을 가질 가능성이 높다고 판단합니다. 즉, 해당 시계열 데이터는 정상성을 가질 가능성이 높다고 할 수 있습니다.
  - p-value가 **0.05보다 작으면** (일반적으로 0.05 또는 0.01보다 작음): 귀무가설을 기각하고, 대립가설을 채택합니다. 이 경우 해당 시계열 데이터는 정상성을 가지지 않는 것으로 간주됩니다.

In [61]:
from statsmodels.tsa.stattools import adfuller,kpss

# 'date' 열을 날짜 형식으로 변환합니다.
df['date'] = pd.to_datetime(df['date'])

# target_column에는 검정을 수행할 시계열 데이터의 컬럼 이름을 지정합니다.
target_column = 'y'

# 'ticker' 열을 기준으로 데이터를 그룹화합니다.
grouped = df.groupby('ticker')

# 수익률로 변환하는 함수를 정의합니다.
def calculate_returns(data):
    returns = data[target_column].pct_change().dropna()
    return returns

tickers = []

for name, group in grouped:
    returns = calculate_returns(group)

    if returns.isnull().any():  # NaN 값이 존재하는 경우
        print(f"Ticker '{name}' contains NaN values.")
        continue

    # KPSS 검정 수행
    result_kpss = kpss(returns)
    p_value_kpss = result_kpss[1]

    print(f"Ticker '{name}' KPSS P-value: {p_value_kpss}")


'''
for name, group in grouped:
    returns = calculate_returns(group)
    returns = returns.dropna()
    print(name, returns)

    # KPSS 검정 수행
    result_kpss = kpss(returns)
    p_value_kpss = result_kpss[1]

    tickers.append((name, p_value_kpss))

print("Tickers with KPSS:")
for ticker, p_value_kpss in tickers:
    print(f"Ticker Name : {ticker}")
    print(f"KPSS P-value : {p_value_kpss}")
    print("-------------------------")
'''

num_tickers = len(tickers)

Ticker 'A000020' KPSS P-value: 0.1
Ticker 'A000040' KPSS P-value: 0.1
Ticker 'A000050' KPSS P-value: 0.1
Ticker 'A000070' KPSS P-value: 0.1
Ticker 'A000080' KPSS P-value: 0.1
Ticker 'A000100' KPSS P-value: 0.1
Ticker 'A000120' KPSS P-value: 0.1
Ticker 'A000140' KPSS P-value: 0.1
Ticker 'A000150' KPSS P-value: 0.1
Ticker 'A000180' KPSS P-value: 0.1
Ticker 'A000210' KPSS P-value: 0.1
Ticker 'A000220' KPSS P-value: 0.1
Ticker 'A000230' KPSS P-value: 0.1
Ticker 'A000240' KPSS P-value: 0.1
Ticker 'A000250' KPSS P-value: 0.1
Ticker 'A000270' KPSS P-value: 0.1
Ticker 'A000300' KPSS P-value: 0.1
Ticker 'A000320' KPSS P-value: 0.1
Ticker 'A000370' KPSS P-value: 0.1
Ticker 'A000390' KPSS P-value: 0.1
Ticker 'A000400' KPSS P-value: 0.1
Ticker 'A000430' KPSS P-value: 0.1
Ticker 'A000440' KPSS P-value: 0.1
Ticker 'A000480' KPSS P-value: 0.1
Ticker 'A000490' KPSS P-value: 0.1
Ticker 'A000500' KPSS P-value: 0.1
Ticker 'A000520' KPSS P-value: 0.1
Ticker 'A000540' KPSS P-value: 0.1
Ticker 'A000590' KPS

In [62]:
from statsmodels.tsa.stattools import adfuller,kpss

# 'date' 열을 날짜 형식으로 변환합니다.
df['date'] = pd.to_datetime(df['date'])

# target_column에는 검정을 수행할 시계열 데이터의 컬럼 이름을 지정합니다.
target_column = 'y'

# 'ticker' 열을 기준으로 데이터를 그룹화합니다.
grouped = df.groupby('ticker')

# 수익률로 변환하는 함수를 정의합니다.
def calculate_returns(data):
    return data[target_column].pct_change().dropna()

# p-value가 0.05보다 작은 경우를 저장할 리스트를 초기화합니다.
non_stationary_tickers = []

# 각 그룹에 대해 ADF 검정과 KPSS 검정을 수행하고, p-value를 확인합니다.
for name, group in grouped:
    returns = calculate_returns(group)

    # ADF 검정 수행
    result_adf = adfuller(returns)
    p_value_adf = result_adf[1]

    # KPSS 검정 수행
    result_kpss = kpss(returns)
    p_value_kpss = result_kpss[1]

    # ADF 검정과 KPSS 검정 중 하나라도 만족하지 않는 경우, 해당 종목을 리스트에 추가합니다.
    if p_value_adf < 0.05 or p_value_kpss < 0.05:
        non_stationary_tickers.append((name, p_value_adf, p_value_kpss))

# p-value가 0.05보다 작은 경우를 출력합니다.
print("Tickers with p-value < 0.05 (Non-stationary):")
for ticker, p_value_adf, p_value_kpss in non_stationary_tickers:
    print(f"Ticker Name : {ticker}")
    print(f"ADF P-value : {p_value_adf}")
    print(f"KPSS P-value : {p_value_kpss}")
    print("-------------------------")

# p-value가 0.05보다 작은 경우가 몇 개의 ticker에서 나오는지 세어 출력합니다.
num_non_stationary_tickers = len(non_stationary_tickers)
print(f"Number of tickers with p-value < 0.05 (Non-stationary): {num_non_stationary_tickers}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
ADF P-value : 4.709167501878699e-16
KPSS P-value : 0.1
-------------------------
Ticker Name : A038110
ADF P-value : 0.0
KPSS P-value : 0.1
-------------------------
Ticker Name : A038290
ADF P-value : 0.0
KPSS P-value : 0.1
-------------------------
Ticker Name : A038390
ADF P-value : 2.6898894808081644e-17
KPSS P-value : 0.1
-------------------------
Ticker Name : A038460
ADF P-value : 3.7929527418743445e-11
KPSS P-value : 0.1
-------------------------
Ticker Name : A038500
ADF P-value : 0.0
KPSS P-value : 0.1
-------------------------
Ticker Name : A038530
ADF P-value : 4.1855434807192384e-23
KPSS P-value : 0.1
-------------------------
Ticker Name : A038540
ADF P-value : 0.0
KPSS P-value : 0.1
-------------------------
Ticker Name : A038620
ADF P-value : 5.247724056237772e-29
KPSS P-value : 0.1
-------------------------
Ticker Name : A038680
ADF P-value : 0.0
KPSS P-value : 0.1
-------------------------
Ticker Name : A038870
ADF P-value : 2.31512